<a href="https://colab.research.google.com/github/anujkadu/InfinityPool-Work/blob/main/Infinity_Pool_TensorFlow_Manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Dataset (will default a loan)

In [59]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Generate synthetic financial dataset
np.random.seed(42)
N = 1000
df = pd.DataFrame({
    'age': np.random.randint(21, 60, N),
    'income': np.random.randint(20000, 150000, N),
    'credit_score': np.random.randint(300, 900, N),
    'loan_amount': np.random.randint(50000, 1000000, N),
    'loan_term': np.random.choice([12, 24, 36, 48, 60], N),
    'past_defaults': np.random.poisson(0.5, N)
})
df['will_default'] = (
    (df['credit_score'] < 600).astype(int) |
    ((df['loan_amount'] / df['income']) > 10).astype(int) |
    (df['past_defaults'] > 2).astype(int)
)

# Convert to tensorflow compatible format

In [60]:
X_raw = df.drop("will_default", axis=1).values
y = df["will_default"].values
scaler = MinMaxScaler()
X = scaler.fit_transform(X_raw)

X_tensor = tf.constant(X, dtype=tf.float32)
y_tensor = tf.constant(y.reshape(-1, 1), dtype=tf.float32)

# Manually declare weights for each neuron of our model

In [61]:
W1 = tf.Variable([
    [0.05, 0.1, 0.2, 0.0, 0.1, 0.3, 0.0, 0.15],
    [0.0, 0.0, 0.1, 0.2, 0.0, 0.2, 0.1, 0.1],
    [0.2, 0.2, 0.0, 0.1, 0.3, 0.0, 0.1, 0.0],
    [0.1, 0.0, 0.0, 0.3, 0.1, 0.0, 0.2, 0.05],
    [0.05, 0.1, 0.1, 0.1, 0.05, 0.1, 0.0, 0.0],
    [0.3, 0.2, 0.1, 0.0, 0.0, 0.1, 0.3, 0.2]
], dtype=tf.float32)
b1 = tf.Variable([0.1, 0.2, 0.0, -0.1, 0.1, 0.0, 0.05, -0.05], dtype=tf.float32)

# Second hidden layer (8 → 4)
W2 = tf.Variable([
    [0.10, 0.20, 0.00, 0.10],
    [0.00, 0.10, 0.20, 0.00],
    [0.20, 0.10, 0.10, 0.00],
    [0.10, 0.00, 0.00, 0.30],
    [0.10, 0.00, 0.30, 0.10],
    [0.00, 0.20, 0.10, 0.00],
    [0.20, 0.10, 0.00, 0.10],
    [0.00, 0.00, 0.10, 0.20]
], dtype=tf.float32)
b2 = tf.Variable([0.00, 0.10, -0.05, 0.20], dtype=tf.float32)

# Output layer (4 → 1)
W3 = tf.Variable([[0.2], [0.1], [-0.1], [0.3]], dtype=tf.float32)
b3 = tf.Variable([0.1], dtype=tf.float32)

# Declare optimizer and loss function

In [62]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Calculate linear transformation and use activation function

In [63]:
Z1 = tf.matmul(X_tensor, W1) + b1
A1 = tf.nn.relu(Z1)

Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)

Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)  # final output: probability

print("Predicted Probabilities (first 5):")
print(A3.numpy()[:5])


Predicted Probabilities (first 5):
[[0.57750833]
 [0.571759  ]
 [0.5744479 ]
 [0.5485885 ]
 [0.5649627 ]]


# Calculate Loss and Accuracy before training

In [64]:
loss = loss_fn(y_tensor, A3)
y_pred_before = tf.cast(A3 > 0.5, dtype=tf.int32)
y_true = tf.cast(y_tensor, dtype=tf.int32)
accuracy_before = tf.reduce_mean(tf.cast(tf.equal(y_pred_before, y_true), tf.float32))

print(f"Loss: {loss.numpy():.4f}")
print(f"Accuracy Before Update: {accuracy_before.numpy() * 100:.4f}%")


Loss: 0.6682
Accuracy Before Update: 63.8000%


# Sample prediction before training

In [65]:
new_customer = pd.DataFrame([{
    'age': 29,
    'income': 25000,
    'credit_score': 490,
    'loan_amount': 450000,
    'loan_term': 60,
    'past_defaults': 3
}])

# Normalize
new_customer_scaled = scaler.transform(new_customer)
X_new = tf.constant(new_customer_scaled, dtype=tf.float32)

# Manual forward pass BEFORE training
Z1 = tf.matmul(X_new, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)

print("BEFORE Training → Probability of default:", A3.numpy()[0][0])


BEFORE Training → Probability of default: 0.5693356


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


# Calculate Gradients

tf.matmul: matrix multiplication

tf.nn.relu: zeroes out negatives (adds non-linearity)

tf.nn.sigmoid: converts raw score into probability between 0 and 1

In [66]:
with tf.GradientTape() as tape:
    Z1 = tf.matmul(X_tensor, W1) + b1
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(A1, W2) + b2
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(A2, W3) + b3
    A3 = tf.nn.sigmoid(Z3)
    loss = loss_fn(y_tensor, A3)

# Compute gradients
grads = tape.gradient(loss, [W1, b1, W2, b2, W3, b3])
print("Gradients calculated for all weights and biases.")


Gradients calculated for all weights and biases.


# 1 Sample Epoch

In [67]:
# Apply gradients to update weights
optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2, W3, b3]))

# Rerun forward pass to see new predictions
Z1 = tf.matmul(X_tensor, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)

# Accuracy after update
y_pred = tf.cast(A3 > 0.5, dtype=tf.int32)
y_true = tf.cast(y_tensor, dtype=tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))

print("Predicted Probabilities After Update (first 5):")
print(A3.numpy()[:5])
print(f"Accuracy After Update: {accuracy.numpy() * 100:.4f}%")

# Print updated weights
print("\nUpdated Weights:")
print("W1:\n", W1.numpy())
print("W2:\n", W2.numpy())
print("W3:\n", W3.numpy())


Predicted Probabilities After Update (first 5):
[[0.59297323]
 [0.58545554]
 [0.588917  ]
 [0.55651724]
 [0.5771877 ]]
Accuracy After Update: 63.8000%

Updated Weights:
W1:
 [[ 0.05998636  0.09009433  0.20997621  0.00999052  0.10995258  0.30990568
   0.00998805  0.15998083]
 [-0.00918652  0.00617352  0.0913417   0.19029479 -0.00763409  0.19382648
   0.09071912  0.09605084]
 [ 0.19000886  0.20993868 -0.00998455  0.09000558  0.2900308  -0.00993868
   0.09000777 -0.00998722]
 [ 0.10999209 -0.00994529  0.00998621  0.30999494  0.10997254  0.00994529
   0.20999308  0.05998865]
 [ 0.05998746  0.09008672  0.10997813  0.10999187  0.05995642  0.10991328
   0.00998902  0.00998212]
 [ 0.30995333  0.19031757  0.10991862  0.00996951  0.00983862  0.10968243
   0.3099591   0.20993479]]
W2:
 [[ 0.10998223  0.20996459 -0.00996459  0.10998812]
 [ 0.00998752  0.10997514  0.19002485  0.00999166]
 [ 0.20998581  0.10997172  0.09002828  0.00999051]
 [ 0.10998176  0.00996365 -0.00996365  0.3099878 ]
 [ 0.10993

# Running multiple epochs

In [68]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Training loop
epochs = 100

for epoch in range(1, epochs + 1):
    with tf.GradientTape() as tape:
        # Forward pass
        Z1 = tf.matmul(X_tensor, W1) + b1
        A1 = tf.nn.relu(Z1)
        Z2 = tf.matmul(A1, W2) + b2
        A2 = tf.nn.relu(Z2)
        Z3 = tf.matmul(A2, W3) + b3
        A3 = tf.nn.sigmoid(Z3)

        # Compute loss
        loss = loss_fn(y_tensor, A3)

    # Compute gradients
    grads = tape.gradient(loss, [W1, b1, W2, b2, W3, b3])
    # Update weights
    optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2, W3, b3]))

    # Compute accuracy
    y_pred = tf.cast(A3 > 0.5, dtype=tf.int32)
    y_true = tf.cast(y_tensor, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))

    # Print progress every 10 epochs
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} → Loss: {loss.numpy():.4f} | Accuracy: {accuracy.numpy() * 100:.2f}%")
    # print("\nUpdated Weights:")
    # print("W1:\n", W1.numpy())
    # print("W2:\n", W2.numpy())
    # print("W3:\n", W3.numpy())

Epoch 001 → Loss: 0.6651 | Accuracy: 63.80%
Epoch 010 → Loss: 0.6482 | Accuracy: 63.80%
Epoch 020 → Loss: 0.6134 | Accuracy: 63.80%
Epoch 030 → Loss: 0.5301 | Accuracy: 73.70%
Epoch 040 → Loss: 0.4206 | Accuracy: 81.60%
Epoch 050 → Loss: 0.3278 | Accuracy: 86.30%
Epoch 060 → Loss: 0.2919 | Accuracy: 86.10%
Epoch 070 → Loss: 0.2871 | Accuracy: 86.40%
Epoch 080 → Loss: 0.2873 | Accuracy: 86.20%
Epoch 090 → Loss: 0.2864 | Accuracy: 86.40%
Epoch 100 → Loss: 0.2859 | Accuracy: 86.40%


# Prediction based on fine tuned model

In [69]:
new_customer = pd.DataFrame([{
    'age': 29,
    'income': 25000,
    'credit_score': 490,
    'loan_amount': 450000,
    'loan_term': 60,
    'past_defaults': 3
}])

# Normalize
new_customer_scaled = scaler.transform(new_customer)
X_new = tf.constant(new_customer_scaled, dtype=tf.float32)

# Manual forward pass BEFORE training
Z1 = tf.matmul(X_new, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)

print("BEFORE Training → Probability of default:", A3.numpy()[0][0])


BEFORE Training → Probability of default: 0.9992848


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
